In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

project_directory = '/Cranio_Lab/Louk_Seton/mesenchyme_project_2023'
os.chdir(os.path.expanduser("~")+project_directory)

In [2]:
### Download opentargets and embl database
# !wget --directory-prefix=GWAS/opentargets/variant_index/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/genetics/22.09/lut/variant-index/
# !wget --directory-prefix=GWAS/opentargets/v2g_scored/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/genetics/22.09/v2g_scored/
# !wget --directory-prefix=GWAS/opentargets/v2g/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/genetics/22.09/v2g/
# !wget --output-document=GWAS/Catalog/gwas_catalog_v1.0-associations.tsv https://www.ebi.ac.uk/gwas/api/search/downloads/full
# !wget --output-document=GWAS/Catalog/gwas_catalog_v1.0.2-associations.tsv https://www.ebi.ac.uk/gwas/api/search/downloads/alternative
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByOverallDirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByOverallIndirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/diseases
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByDatatypeDirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByDatatypeIndirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByDatasourceIndirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/associationByDatasourceDirect
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/targets
# !wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/24.06/output/etl/parquet/evidence/sourceId=ot_genetics_portal/
#!wget --directory-prefix=GWAS/opentargets/platform/ --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/genetics/latest/v2g_scored/

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

In [4]:
spark = SparkSession.builder \
           .getOrCreate()

25/08/13 10:05:17 WARN Utils: Your hostname, cranio213 resolves to a loopback address: 127.0.1.1; using 172.16.4.213 instead (on interface enp1s0f0)
25/08/13 10:05:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/13 10:05:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
ot_genetics = "GWAS/opentargets/"
ot_platform = "GWAS/opentargets/platform/"
variant_index = spark.read.parquet(ot_genetics+"variant_index/")
v2g_scored = spark.read.parquet(ot_genetics+"v2g_scored/")
v2g = spark.read.parquet(ot_genetics+"v2g/")
associationByOverallDirect = spark.read.parquet(ot_platform+"associationByOverallDirect/")
associationByOverallIndirect = spark.read.parquet(ot_platform+"associationByOverallIndirect/")
associationByDatatypeDirect = spark.read.parquet(ot_platform+"associationByDatatypeDirect/")
associationByDatatypeIndirect = spark.read.parquet(ot_platform+"associationByDatatypeIndirect/")
associationByDatasourceIndirect = spark.read.parquet(ot_platform+"associationByDatasourceIndirect/")
associationByDatasourceDirect = spark.read.parquet(ot_platform+"associationByDatasourceDirect/")
evidence_ot_genetics_portal = spark.read.parquet(ot_platform+"evidence/sourceId=ot_genetics_portal/")
diseases = spark.read.parquet(ot_platform+"diseases/")
targets = spark.read.parquet(ot_platform+"targets/")

In [6]:
evidence_ot_genetics_portal = spark.read.parquet(ot_platform+"evidence/sourceId=ot_genetics_portal/")

In [66]:
associationByOverallIndirect.persist()
#count total
(associationByOverallIndirect
      .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
      .orderBy('score', ascending = False) #reorder by score before removing duplicates
      .dropDuplicates(['targetId'])
    .count())

#keep all
list((associationByOverallIndirect
      .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
      .orderBy('score', ascending = False) #reorder by score before removing duplicates
      .dropDuplicates(['targetId']) #drop duplicates
      .orderBy('score', ascending = False) #?order again
      .select('targetId')
      .toPandas()
)['targetId'])

#filter by score
list((associationByOverallIndirect
      .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
      .orderBy('score', ascending = False) #reorder by score before removing duplicates
      .dropDuplicates(['targetId']) #drop duplicates
      .filter(col("score")>0.4)
      .orderBy('score', ascending = False) #?order again
      .select('targetId')
      .toPandas()      
)['targetId'])

#filter by top n genes
list((associationByOverallIndirect
      .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
      .orderBy('score', ascending = False) #reorder by score before removing duplicates
      .dropDuplicates(['targetId']) #drop duplicates
      .orderBy('score', ascending = False) #?order again
      .limit(10)
      .select('targetId')
      .toPandas()      
)['targetId'])

associationByOverallIndirect.unpersist()

629

In [7]:
#gwas query function
#Supply a disease code, check the database for associated genes. Cutoff either by score or by top n genes. Can use overall score, limit to results from specific datasource, 
#use database score. List of genes is human, needs to be moved to mouse ortholog.

###code step by step

#done - check if OT database is present, if not error
#done - Check if disease code exists in database, if not report error
#done - check if datasource exists (if not using overall score), if not report error
    #either direct or indirect
#done - Include disease children yes/no ###be sure to include original term as well
    #if yes, report the codes of the children
#done - Sort by score of datasource of choice
#done - drop duplicates in correct order
#done - report total unique genes found
#compare with mouse, find orthologs
    #report number of orthologs found
    #if we want the gene name, change ensemblid to gene name
    #########################
    #this part needs some work, need to properly drop NA only in the ID column and replace missing gene names with ID. Now we might toss genes that do map.
    #need to write a function to retrieve ensembl orthologs by itself
    #need to write a case for human genes to skip orthologs but still need to map them to gene names if wanted

#Filter by selected method, (all, score cut-off (between 0-1), top n genes)
#return list of genes (either by ensembl id or gene name)

############
#once finished I need to rewrite the code and take out some of the subfunctions for readability and easy understanding
#should create a class with all the required subfunctions


#### import libraries ####
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from typing import Union

def OT_genes_by_disease(
    disease_ID: str = 'MONDO_0016064', #disease id, set default to cleft palate
    gene_cutoff: Union[int, float, str] = 'All', #All for all genes, cutoff score between 0-1, or top n genes cutoff
    datasource: list = ['overall'], #somehow check if datasource exists, default uses overall. list of datasources
    evidence_type: str = 'indirect', #whether to use direct or indirect evidence
    database_location: str = 'GWAS/opentargets/platform/', #location of the database
    include_descendants: bool = True, #whether or not to include disease descendants
    ortholog_map: str = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt', #Location of csv file mapping human genes to focal species. Expects columns in the following order, Human gene stable ID, Human gene name, Focal species stable ID, focal species gene name.
    return_gene_names: bool = True, #whether to return gene names or ensemblid

    
):   
    #some checks on variables
    datasource_list = ['overall','europepmc', 'crispr', 'uniprot_variants', 'genomics_england', 'gene_burden', 'chembl', 'slapenrich', 'crispr_screen', 
                       'expression_atlas','intogen', 'reactome', 'orphanet', 'sysbio', 'cancer_biomarkers', 'cancer_gene_census', 'progeny', 'clingen',
                       'eva', 'gene2phenotype', 'uniprot_literature','eva_somatic', 'ot_genetics_portal', 'impc']
    if datasource not in datasource_list:
        if not set(datasource).issubset(datasource_list):
            raise ValueError("Datasource does not exist. Make sure datasource is 'overall' or a valid datasource.")
    
    evidence_type_list = ['direct', 'indirect']
    if evidence_type not in evidence_type_list:
        raise ValueError("evidence_type needs to be either 'direct' or 'indirect'.")
    
    #test if database directory exists
    if not os.path.isdir(database_location):
        raise ValueError('Database directory does not exist')
    
    #start spark session
    spark = SparkSession.builder \
            .getOrCreate()
    
    #load database
    try:
        diseases = spark.read.parquet(database_location+"diseases/") #load diseases reference
        diseases.persist() #persist diseases for quicker loading
        #now we want to load the correct database
        datasource_filter = False
        if evidence_type=='indirect': #if we want indirect evidence
            if datasource==['overall']: #if we want to look at overall scores
                spark_association_database = spark.read.parquet(database_location+"associationByOverallIndirect/")
            else: #if we want to look at other datasource scores
                datasource_filter = True                
                spark_association_database_pre = spark.read.parquet(database_location+"associationByDatasourceIndirect/") #name it pre so we can filter afterwards and use the same code
        elif evidence_type=='direct':#if we want direct evidence
            if datasource==['overall']: 
                spark_association_database = spark.read.parquet(database_location+"associationByOverallDirect/")
            else:
                datasource_filter = True
                spark_association_database_pre = spark.read.parquet(database_location+"associationByDatasourceDirect/")
        else: raise Exception('Unknown error loading database')#should raise but put it there in case    
        # associationByDatatypeDirect = spark.read.parquet(database_location+"associationByDatatypeDirect/") #we are not including datatypes yet
        # associationByDatatypeIndirect = spark.read.parquet(database_location+"associationByDatatypeIndirect/")
    except ValueError:
        print('error loading database, make sure the database location contains the required files')
    
    #check if disease ID exists
    if not bool(diseases.filter(col("id")==disease_ID).collect()):
        raise ValueError('Disease ID does not exist')
    
    #create disease list
    disease_list = [disease_ID] #disease list starts with the disease ID
    if include_descendants: #if descendants are to be included
        print('including descendants')
        descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
        print('found', len(descendant_list), 'descendants')
        disease_list = disease_list + descendant_list #concatenate two lists
        print('list of disease_IDs:',disease_list)
    
    #Get list of genes for diseases
    if datasource_filter is True:
        spark_association_database = spark_association_database_pre.filter(col("datasourceId").isin(datasource))
        print('filtering by datasource list:',datasource)
        spark_association_database.persist() #persist for quicker loading
    else:
        spark_association_database.persist()
    
    #getting list of genes
    total_genes = (spark_association_database
                           .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                           .orderBy('score', ascending = False) #reorder by score before removing duplicates
                           .dropDuplicates(['targetId'])
                           .count()
                          )
    print(total_genes, 'genes found')
    if gene_cutoff == 'all' or gene_cutoff == 'All': #keep all genes
        gene_list = list((spark_association_database
                          .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                          .orderBy('score', ascending = False) #reorder by score before removing duplicates
                          .dropDuplicates(['targetId']) #drop duplicates
                          .orderBy('score', ascending = False) #?order again
                          .select('targetId')
                          .toPandas()
                         )['targetId'])
        print('keeping all genes')
    elif isinstance(gene_cutoff, float): #score cutoff filter
        if (gene_cutoff>=0)&(gene_cutoff<=1): #make sure the score cutoff is between 0 and 1
            gene_list = list((spark_association_database
                              .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                              .orderBy('score', ascending = False) #reorder by score before removing duplicates
                              .dropDuplicates(['targetId']) #drop duplicates
                              .filter(col("score")>=gene_cutoff)
                              .orderBy('score', ascending = False) #?order again
                              .select('targetId')
                              .toPandas()      
                             )['targetId'])
            print('filtering scores larger than', gene_cutoff)
            print('keeping', len(gene_list), 'genes')
        else: raise Exception("gene_cutoff needs to be 'All', a float between 0 and 1 or an int")
    elif isinstance(gene_cutoff, int): #top n genes filter
        gene_list = list((spark_association_database
                          .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                          .orderBy('score', ascending = False) #reorder by score before removing duplicates
                          .dropDuplicates(['targetId']) #drop duplicates
                          .orderBy('score', ascending = False) #?order again
                          .limit(gene_cutoff)
                          .select('targetId')
                          .toPandas()      
                       )['targetId'])
        print('filtering top', gene_cutoff, 'genes')
        print('keeping', len(gene_list), 'genes')
    else: raise Exception('Unknown error filtering genes') #idk what would cause this to raise but I put it there in case
    
    diseases.unpersist() #unload data from cache
    spark_association_database.unpersist()

    #map orthologues
    if ortholog_map is not None:
        print('mapping using specified orthologue file')
        ortholog_map_df = pd.read_csv(ortholog_map)
        ortholog_map_df = ortholog_map_df.dropna()
        input_gene_count = len(gene_list)
        if return_gene_names is True:
            ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
        else:
            ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,2]))
        gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
        print(len(gene_list), 'focal species genes mapped to', input_gene_count, 'human genes.')
        
    return(gene_list)

In [8]:
#gwas query function
#Supply a disease code, check the database for associated genes. Cutoff either by score or by top n genes. Can use overall score, limit to results from specific datasource, 
#use database score. List of genes is human, needs to be moved to mouse ortholog.

###code step by step

#done - check if OT database is present, if not error
#done - Check if disease code exists in database, if not report error
#done - check if datasource exists (if not using overall score), if not report error
    #either direct or indirect
#done - Include disease children yes/no ###be sure to include original term as well
    #if yes, report the codes of the children
#done - Sort by score of datasource of choice
#done - drop duplicates in correct order
#done - report total unique genes found
#compare with mouse, find orthologs
    #report number of orthologs found
    #if we want the gene name, change ensemblid to gene name
    #########################
    #this part needs some work, need to properly drop NA only in the ID column and replace missing gene names with ID. Now we might toss genes that do map.
    #need to write a function to retrieve ensembl orthologs by itself
    #need to write a case for human genes to skip orthologs but still need to map them to gene names if wanted

#Filter by selected method, (all, score cut-off (between 0-1), top n genes)
#return list of genes (either by ensembl id or gene name)

############
#once finished I need to rewrite the code and take out some of the subfunctions for readability and easy understanding
#should create a class with all the required subfunctions


#### import libraries ####
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from typing import Union

def OT_genes_by_disease(
    disease_ID: str = 'MONDO_0016064', #disease id, set default to cleft palate
    gene_cutoff: Union[int, float, str] = 'All', #All for all genes, cutoff score between 0-1, or top n genes cutoff
    datasource: list = ['overall'], #somehow check if datasource exists, default uses overall. list of datasources
    evidence_type: str = 'indirect', #whether to use direct or indirect evidence
    database_location: str = 'GWAS/opentargets/platform/', #location of the database
    include_descendants: bool = True, #whether or not to include disease descendants
    ortholog_map: str = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt', #Location of csv file mapping human genes to focal species. Expects columns in the following order, Human gene stable ID, Human gene name, Focal species stable ID, focal species gene name.
    return_gene_names: bool = True, #whether to return gene names or ensemblid

    
):   
    #some checks on variables
    datasource_list = ['overall','europepmc', 'crispr', 'uniprot_variants', 'genomics_england', 'gene_burden', 'chembl', 'slapenrich', 'crispr_screen', 
                       'expression_atlas','intogen', 'reactome', 'orphanet', 'sysbio', 'cancer_biomarkers', 'cancer_gene_census', 'progeny', 'clingen',
                       'eva', 'gene2phenotype', 'uniprot_literature','eva_somatic', 'ot_genetics_portal', 'impc']
    if datasource not in datasource_list:
        if not set(datasource).issubset(datasource_list):
            raise ValueError("Datasource does not exist. Make sure datasource is 'overall' or a valid datasource.")
    
    evidence_type_list = ['direct', 'indirect']
    if evidence_type not in evidence_type_list:
        raise ValueError("evidence_type needs to be either 'direct' or 'indirect'.")
    
    #test if database directory exists
    if not os.path.isdir(database_location):
        raise ValueError('Database directory does not exist')
    
    #start spark session
    spark = SparkSession.builder \
            .getOrCreate()
    
    #load database
    try:
        diseases = spark.read.parquet(database_location+"diseases/") #load diseases reference
        diseases.persist() #persist diseases for quicker loading
        #now we want to load the correct database
        datasource_filter = False
        if evidence_type=='indirect': #if we want indirect evidence
            if datasource==['overall']: #if we want to look at overall scores
                spark_association_database = spark.read.parquet(database_location+"associationByOverallIndirect/")
            else: #if we want to look at other datasource scores
                datasource_filter = True                
                spark_association_database_pre = spark.read.parquet(database_location+"associationByDatasourceIndirect/") #name it pre so we can filter afterwards and use the same code
        elif evidence_type=='direct':#if we want direct evidence
            if datasource==['overall']: 
                spark_association_database = spark.read.parquet(database_location+"associationByOverallDirect/")
            else:
                datasource_filter = True
                spark_association_database_pre = spark.read.parquet(database_location+"associationByDatasourceDirect/")
        else: raise Exception('Unknown error loading database')#should raise but put it there in case    
        # associationByDatatypeDirect = spark.read.parquet(database_location+"associationByDatatypeDirect/") #we are not including datatypes yet
        # associationByDatatypeIndirect = spark.read.parquet(database_location+"associationByDatatypeIndirect/")
    except ValueError:
        print('error loading database, make sure the database location contains the required files')
    
    #check if disease ID exists
    if not bool(diseases.filter(col("id")==disease_ID).collect()):
        raise ValueError('Disease ID does not exist')
    
    #create disease list
    disease_list = [disease_ID] #disease list starts with the disease ID
    if include_descendants: #if descendants are to be included
        print('including descendants')
        descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
        print('found', len(descendant_list), 'descendants')
        disease_list = disease_list + descendant_list #concatenate two lists
        print('list of disease_IDs:',disease_list)
    
    #Get list of genes for diseases
    if datasource_filter is True:
        spark_association_database = spark_association_database_pre.filter(col("datasourceId").isin(datasource))
        print('filtering by datasource list:',datasource)
        spark_association_database.persist() #persist for quicker loading
    else:
        spark_association_database.persist()
    
    #getting list of genes
    total_genes = (spark_association_database
                           .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                           .orderBy('score', ascending = False) #reorder by score before removing duplicates
                           .dropDuplicates(['targetId'])
                           .count()
                          )
    print(total_genes, 'genes found')
    if gene_cutoff == 'all' or gene_cutoff == 'All': #keep all genes
        gene_list_df = (spark_association_database
                          .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                          .orderBy('score', ascending = False) #reorder by score before removing duplicates
                          .dropDuplicates(['targetId']) #drop duplicates
                          .orderBy('score', ascending = False) #?order again
                          .select('targetId','score')
                          .toPandas()
                         )
        print('keeping all genes')
    elif isinstance(gene_cutoff, float): #score cutoff filter
        if (gene_cutoff>=0)&(gene_cutoff<=1): #make sure the score cutoff is between 0 and 1
            gene_list_df = (spark_association_database
                              .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                              .orderBy('score', ascending = False) #reorder by score before removing duplicates
                              .dropDuplicates(['targetId']) #drop duplicates
                              .filter(col("score")>=gene_cutoff)
                              .orderBy('score', ascending = False) #?order again
                              .select('targetId','score')
                              .toPandas()      
                             )
            print('filtering scores larger than', gene_cutoff)
            print('keeping', len(gene_list_df), 'genes')
        else: raise Exception("gene_cutoff needs to be 'All', a float between 0 and 1 or an int")
    elif isinstance(gene_cutoff, int): #top n genes filter
        gene_list_df = (spark_association_database
                          .filter(col("diseaseId").isin(disease_list))  #filter by disease ID
                          .orderBy('score', ascending = False) #reorder by score before removing duplicates
                          .dropDuplicates(['targetId']) #drop duplicates
                          .orderBy('score', ascending = False) #?order again
                          .limit(gene_cutoff)
                          .select('targetId','score')
                          .toPandas()      
                       )
        print('filtering top', gene_cutoff, 'genes')
        print('keeping', len(gene_list_df), 'genes')
    else: raise Exception('Unknown error filtering genes') #idk what would cause this to raise but I put it there in case
    
    diseases.unpersist() #unload data from cache
    spark_association_database.unpersist()

    #map orthologues
    if ortholog_map is not None:
        print('mapping using specified orthologue file')
        ortholog_map_df = pd.read_csv(ortholog_map)
        ortholog_map_df = ortholog_map_df.dropna()
        gene_list = list(gene_list_df['targetId'])
        input_gene_count = len(gene_list)
        if return_gene_names is True:
            ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
        else:
            ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,2]))
        #gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
        gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)
        gene_list_df.dropna(inplace=True)
        print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')
        gene_list_df = gene_list_df.to_dict()
        #gene_list_df = gene_list_df[gene_list_df['targetId'].isin(gene_list)]
    return(gene_list_df)

In [107]:
trait_gene_dict = OT_genes_by_disease(disease_ID = 'MONDO_0023369',
                                             datasource = ['overall'],
                                             gene_cutoff = 0.2,)
list(trait_gene_dict['targetId'].values())

including descendants
found 63 descendants
list of disease_IDs: ['MONDO_0023369', 'MONDO_0021580', 'MONDO_0004328', 'EFO_0007356', 'MONDO_0024611', 'MONDO_0002580', 'MONDO_0029145', 'MONDO_0002186', 'EFO_1000209', 'MONDO_0015411', 'MONDO_0012142', 'EFO_1001818', 'MONDO_0015416', 'MONDO_0015417', 'EFO_1000365', 'MONDO_0016064', 'MONDO_0015496', 'Orphanet_3253', 'MONDO_0010560', 'MONDO_0001122', 'EFO_0007264', 'MONDO_0000110', 'MONDO_0001123', 'MONDO_0004312', 'MONDO_0008637', 'EFO_1001077', 'MONDO_0002579', 'MONDO_0000358', 'MONDO_0004757', 'MONDO_0004394', 'MONDO_0017627', 'MONDO_0015480', 'EFO_1000731', 'MONDO_0015481', 'MONDO_0013300', 'EFO_0007333', 'MONDO_0008866', 'MONDO_0004943', 'MONDO_0004384', 'MONDO_0007336', 'MONDO_0021484', 'MONDO_0010906', 'MONDO_0012165', 'MONDO_0014388', 'MONDO_0016043', 'MONDO_0016044', 'EFO_0007408', 'EFO_1001035', 'EFO_0700035', 'MONDO_0013378', 'MONDO_0004810', 'MONDO_0004457', 'MONDO_0007927', 'EFO_0007489', 'MONDO_0001624', 'MONDO_0021412', 'EFO_00

1299 genes found
filtering scores larger than 0.2
keeping 59 genes
mapping using specified orthologue file
58 focal species genes mapped to 59 human genes.


58

In [95]:
###running on dict of traits, produce dict of genes
#save as json
trait_list = {'cleft_lip' :'EFO_0003959',
              'cleft_palate' : 'HP_0000175',
              'lip_morphology_measurement' : 'EFO_0007845',
              'chin_morphology_measurement' : 'EFO_0007842',
              'nose_morphology_measurement' : 'EFO_0007843',
              'tooth_agenesis' : 'EFO_0005410',
              'facial_morphology_measurement':'EFO_0007841',}

trait_gene_dict = {}
for i in trait_list:
    trait_gene_dict[i] = OT_genes_by_disease(disease_ID = trait_list[i],
                                             datasource = ['overall'],
                                             gene_cutoff = 'all',)
    # if len(trait_gene_dict[i]['targetId']) < 50:
    #     trait_gene_dict[i] = OT_genes_by_disease(disease_ID = trait_list[i],
    #                                              datasource = ['overall'],
    #                                              gene_cutoff = 50,)

import json
with open('GWAS/trait_json/test_traits_4.json', 'w',encoding='utf-8') as f: 
    json.dump(trait_gene_dict, f, ensure_ascii=False, indent=4)

including descendants
found 2 descendants
list of disease_IDs: ['EFO_0003959', 'HP_0000204', 'HP_0000161']


455 genes found
keeping all genes
mapping using specified orthologue file
433 focal species genes mapped to 455 human genes.
including descendants
found 0 descendants
list of disease_IDs: ['HP_0000175']


759 genes found
keeping all genes
mapping using specified orthologue file
728 focal species genes mapped to 759 human genes.
including descendants
found 0 descendants
list of disease_IDs: ['EFO_0007845']


67 genes found
keeping all genes
mapping using specified orthologue file
66 focal species genes mapped to 67 human genes.
including descendants
found 0 descendants
list of disease_IDs: ['EFO_0007842']


80 genes found
keeping all genes
mapping using specified orthologue file
78 focal species genes mapped to 80 human genes.
including descendants
found 0 descendants
list of disease_IDs: ['EFO_0007843']


158 genes found
keeping all genes
mapping using specified orthologue file
152 focal species genes mapped to 158 human genes.
including descendants
found 5 descendants
list of disease_IDs: ['EFO_0005410', 'MONDO_0007129', 'MONDO_0014749', 'MONDO_0007881', 'MONDO_0010741', 'MONDO_0011265']


339 genes found
keeping all genes
mapping using specified orthologue file
321 focal species genes mapped to 339 human genes.
including descendants
found 14 descendants
list of disease_IDs: ['EFO_0007841', 'EFO_0007842', 'EFO_0007843', 'EFO_0009360', 'EFO_0010949', 'EFO_0010948', 'EFO_0009892', 'EFO_0007844', 'EFO_0007955', 'EFO_0007856', 'EFO_0007845', 'EFO_0007855', 'EFO_0007858', 'EFO_0007857', 'EFO_0007847']


542 genes found
keeping all genes
mapping using specified orthologue file
524 focal species genes mapped to 542 human genes.


In [202]:
import json
with open('GWAS/trait_json/test_traits_2.json', 'w',encoding='utf-8') as f: 
    json.dump(trait_gene_dict, f, ensure_ascii=False, indent=4)

In [ ]:
gene_list_df = evidence_ot_genetics_portal.filter(col("studyId").isin(list(studyID_dict.keys()))).filter(col("score")>min_score).select(['targetId','variantRsId','studyId','score','diseaseFromSourceMappedId']).toPandas()
gene_list_df['area'] = gene_list_df['studyId'].map(area_dict)
gene_list_df

In [92]:
ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df = ortholog_map_df.dropna()
gene_list = list(gene_list_df['targetId'])
input_gene_count = len(gene_list)
ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)

#gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
gene_list_df = gene_list_df[gene_list_df['targetId'].notna()]
print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')
gene_list_df

90 focal species genes mapped to 94 human genes.


,targetId,variantRsId,studyId,score,diseaseFromSourceMappedId,area
0,Tmem74,rs10089785,GCST90007253,0.690749,EFO_0007843,Nose
1,Nav3,rs2695152,GCST90007215,0.656231,EFO_0007845,Upper_lip
2,Gm525,rs227727,GCST90007218,0.346529,EFO_0007845,Upper_lip
3,Pkdcc,rs10189338,GCST90007191,0.714584,EFO_0007843,Nose
4,Hgf,rs7806852,GCST90007248,0.700607,EFO_0007843,Nose
...,...,...,...,...,...,...
89,Has2,rs2581548,GCST90007245,0.483594,EFO_0007841,Frontonasal
90,Runx2,rs73457129,GCST90007264,0.788357,EFO_0007843,Nose
91,Sox9,rs9302943,GCST90007224,0.744489,EFO_0007843,Nose
92,Six2,rs35395759,GCST90007224,0.627206,EFO_0007843,Nose


In [93]:
gene_list_df.sort_values('studyId').to_csv('White_JD_genes_df.csv',index = False)

In [14]:
disease_ID = 'EFO_0007841' #MONDO_0023369
min_score = 0.1

disease_list = [disease_ID] #disease list starts with the disease ID
print('including descendants')
descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
print('found', len(descendant_list), 'descendants')
disease_list = disease_list + descendant_list #concatenate two lists
gene_list_df = evidence_ot_genetics_portal.filter(col("diseaseFromSourceMappedId").isin(disease_list)).filter(col("score")>min_score).select(['targetId','variantRsId','studyId','score','diseaseFromSourceMappedId']).toPandas()

including descendants


found 14 descendants


In [26]:
ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df

,Gene stable ID,Gene name,Mouse gene stable ID,Mouse gene name
0,ENSG00000210049,MT-TF,NaN,NaN
1,ENSG00000211459,MT-RNR1,NaN,NaN
2,ENSG00000210077,MT-TV,NaN,NaN
3,ENSG00000210082,MT-RNR2,NaN,NaN
4,ENSG00000209082,MT-TL1,NaN,NaN
...,...,...,...,...
77642,ENSG00000232679,LINC01705,NaN,NaN
77643,ENSG00000200033,RNU6-403P,ENSMUSG00000088001,Gm22883
77644,ENSG00000228437,LINC02474,NaN,NaN
77645,ENSG00000229463,LYST-AS1,NaN,NaN


In [22]:
disease_ID = 'EFO_0007841' #MONDO_0023369
min_score = 0.4

disease_list = [disease_ID] #disease list starts with the disease ID
print('including descendants')
descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
print('found', len(descendant_list), 'descendants')
disease_list = disease_list + descendant_list #concatenate two lists
gene_list_df = evidence_ot_genetics_portal.filter(col("diseaseFromSourceMappedId").isin(disease_list)).filter(col("score")>min_score).select(['targetId','variantRsId','studyId','score','diseaseFromSourceMappedId']).toPandas()
ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df = ortholog_map_df.dropna()
gene_list = list(gene_list_df['targetId'])
input_gene_count = len(gene_list)
ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)

#gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
gene_list_df = gene_list_df[gene_list_df['targetId'].notna()]
print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')
gene_list_df

including descendants
found 14 descendants


[Stage 31:================================>                      (31 + 22) / 53]

356 focal species genes mapped to 372 human genes.


,targetId,variantRsId,studyId,score,diseaseFromSourceMappedId
0,Cped1,rs884373,GCST90007206,0.811751,EFO_0010948
1,Cped1,rs6950680,GCST90026584,0.788809,EFO_0010948
2,Bmp2,rs3920540,GCST006517,0.439235,EFO_0007843
3,Tmem74,rs10089785,GCST90007253,0.690749,EFO_0007843
4,Alx4,rs7121535,GCST90007181,0.867678,EFO_0007841
...,...,...,...,...,...
367,Six2,rs921120,GCST90026563,0.761771,EFO_0007843
368,Six2,rs35395759,GCST90007224,0.627206,EFO_0007843
369,Eefsec,rs7373685,GCST90007261,0.488346,EFO_0007845
370,Fgf9,rs73168105,GCST003989,0.455364,EFO_0007842


In [40]:
disease_dict = {
    'EFO_0007841': 'facial morphology measurement(GWAS)',
    'HP_0000202': 'orofacial cleft(GWAS)',
    'HP_0000366': 'abnormality of the nose',
    'MONDO_0000358': 'orofacial cleft',
    'HP_0001999': 'abnormal of facial shape',
    'HP_0000153': 'abnormality of the mouth',
    'HP_0000164': 'abnormality of dentition',
    'MONDO_0016643': 'frontonasal dysplasia',
    'EFO_1000720': 'keratosis',
    'EFO_0005917': 'generalised epilepsy',
    'MONDO_0003939': 'muscle tissue disorder',
    'EFO_0000538':'hypertrophic cardiomyopathy',
    'EFO_0000318':'cardiomyopathy',
    'MONDO_0020120':'skeletal muscle disorder',
    'MONDO_0019952':'congenital myopathy',
    'EFO_0000540':'immune system disease',
    'EFO_0000407':'dilated cardiomyopathy(GWAS)',
    'EFO_0006941':'grip strength measurement(GWAS)',
    'EFO_0004264':'vascular disease',
    'EFO_0004695':'intraocular pressure',
    'EFO_0008003':'heart rate variability measurement(GWAS)',
    'EFO_0006336':'diastolic blood pressure',
    'MONDO_0005180':'parkinsons disease',
    'MONDO_0020127':'hereditary peripheral neuropathy',
    'MONDO_0015626':'Charcot-Marie-Tooth disease',
    'MONDO_0019548':'autosomal dominant intermediate Charcot-Marie-Tooth disease',
    'MONDO_0024237':'inherited neurodegenerative disorder',
    'MONDO_0100172':'intellectual disability, autosomal dominant',
    'EFO_0005772':'neurodegenerative disease',
    'EFO_0010642':'neurodevelopmental disorder',
    'EFO_0004875':'mathematical ability',
    'MONDO_0023369':'disorder of facial skeleton',
    'HP_0011821':'abnormal facial skeleton morphology',
    'HP_0000271':'abnormality of the face',
    'MONDO_0015411': 'facial cleft',
    'HP_0001742':'nasal congestion',
}


    

In [41]:
disease_ID = 'HP_0001742' #MONDO_0023369
min_score = 0.4

disease_list = [disease_ID] #disease list starts with the disease ID
print('including descendants')
descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
print('found', len(descendant_list), 'descendants')
disease_list = disease_list + descendant_list #concatenate two lists
disease_list

gene_list_df = associationByOverallDirect.filter(col("diseaseId").isin(disease_list)).filter(col("score")>min_score).select(['targetId','diseaseId','score']).toPandas()

ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df = ortholog_map_df.dropna()
gene_list = list(gene_list_df['targetId'])
input_gene_count = len(gene_list)
ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)

#gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
gene_list_df = gene_list_df[gene_list_df['targetId'].notna()]
print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')

gene_list_df.sort_values('diseaseId').to_csv(disease_ID + '_genes_df.csv',index = False)
gene_list_df

including descendants
found 0 descendants
20 focal species genes mapped to 20 human genes.


,targetId,diseaseId,score
0,Adrb1,HP_0001742,0.540921
1,Slc6a2,HP_0001742,0.603508
2,Trpa1,HP_0001742,0.565428
3,Nr3c1,HP_0001742,0.588986
4,Adra1a,HP_0001742,0.606803
5,Avpr2,HP_0001742,0.564504
6,Chrm3,HP_0001742,0.565059
7,Trpm8,HP_0001742,0.564504
8,Sigmar1,HP_0001742,0.606034
9,Adra2a,HP_0001742,0.600616


In [46]:
disease_ID = 'EFO_0006336' #MONDO_0023369
min_score = 0.6

disease_list = [disease_ID] #disease list starts with the disease ID
print('including descendants')
descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
print('found', len(descendant_list), 'descendants')
disease_list = disease_list + descendant_list #concatenate two lists
gene_list_df = evidence_ot_genetics_portal.filter(col("diseaseFromSourceMappedId").isin(disease_list)).filter(col("score")>min_score).select(['targetId','variantRsId','studyId','score','diseaseFromSourceMappedId']).toPandas()
ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df = ortholog_map_df.dropna()
gene_list = list(gene_list_df['targetId'])
input_gene_count = len(gene_list)
ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)

#gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
gene_list_df = gene_list_df[gene_list_df['targetId'].notna()]
print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')
gene_list_df.sort_values('diseaseFromSourceMappedId').to_csv(disease_ID+'_GWAS_genes_df.csv',index = False)
gene_list_df

including descendants
found 0 descendants


1325 focal species genes mapped to 1369 human genes.


,targetId,variantRsId,studyId,score,diseaseFromSourceMappedId
0,Clic4,rs6686889,GCST004280,0.699869,EFO_0006336
1,Clic4,rs4601530,GCST90000059,0.770126,EFO_0006336
2,Clic4,rs5773100,NEALE2_4079_raw,0.747710,EFO_0006336
3,Clic4,rs4543739,GCST90018952,0.714260,EFO_0006336
4,Fes,rs2521501,GCST001228,0.751210,EFO_0006336
...,...,...,...,...,...
1364,Zbtb10,rs6990531,GCST90018952,0.771132,EFO_0006336
1365,Zbtb10,rs11993814,NEALE2_4079_raw,0.838807,EFO_0006336
1366,Zbtb10,rs150539196,GCST90025981,0.827740,EFO_0006336
1367,Col4a2,rs9559779,NEALE2_4079_raw,0.842621,EFO_0006336


In [8]:
disease_ID = 'MONDO_0003939' #MONDO_0023369
min_score = 0.2

disease_list = [disease_ID] #disease list starts with the disease ID
print('including descendants')
descendant_list = diseases.filter(col("id")==disease_ID).select('descendants',).first()[0] #list of descendants for specific disease id
print('found', len(descendant_list), 'descendants')
disease_list = disease_list + descendant_list #concatenate two lists
disease_list

gene_list_df = associationByOverallDirect.filter(col("diseaseId").isin(disease_list)).filter(col("score")>min_score).select(['targetId','diseaseId','score']).toPandas()

ortholog_map = 'GWAS/ensembl_human_orthologues/Human_to_Mouse.txt'

ortholog_map_df = pd.read_csv(ortholog_map)
ortholog_map_df = ortholog_map_df.dropna()
gene_list = list(gene_list_df['targetId'])
input_gene_count = len(gene_list)
ortholog_map_dict = dict(zip(ortholog_map_df.iloc[:,0], ortholog_map_df.iloc[:,3]))
gene_list_df['targetId'] = gene_list_df['targetId'].map(ortholog_map_dict)

#gene_list = [x for x in map(ortholog_map_dict.get, gene_list) if x is not None]
gene_list_df = gene_list_df[gene_list_df['targetId'].notna()]
print(len(gene_list_df), 'focal species genes mapped to', input_gene_count, 'human genes.')
gene_list_df

including descendants
found 660 descendants


2473 focal species genes mapped to 2602 human genes.


,targetId,diseaseId,score
0,Myh7,EFO_0002945,0.530581
1,Tpm1,EFO_0002945,0.370596
2,Lmna,EFO_0002945,0.688291
3,Acta1,MONDO_0100084,0.539913
4,Get3,MONDO_0859358,0.452735
...,...,...,...
2597,Sdha,MONDO_0100294,0.801257
2598,Sdhb,MONDO_0100294,0.398199
2599,Sdhd,MONDO_0100294,0.687092
2600,Sdhaf1,MONDO_0100294,0.743727


In [9]:
gene_list_df.sort_values('diseaseId').to_csv(disease_ID + '_genes_df.csv',index = False)